
**Nama** : Zidan Muhammad Ikvan,
**Cohort ID** : MC404D5Y0059,
**Email** : zidanikvan@gmail.com

## Import Library/Package

In [1]:
!pip install google-play-scraper pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import time
import pandas as pd
from google_play_scraper import Sort, reviews
from tqdm import tqdm

## Scraping Dataset

In [ ]:
all_reviews = []
token = None
target_total = 11000

print("Mulai scraping data...")

with tqdm(total=target_total) as pbar:
    while len(all_reviews) < target_total:
        result, token = reviews(
            'com.shopee.id',
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=200,
            continuation_token=token
        )
        all_reviews.extend(result)
        pbar.update(len(result))
        time.sleep(1)  # menghindari diblokir oleh server

# Konversi ke DataFrame
df = pd.DataFrame(all_reviews)

# Hapus duplikat berdasarkan isi review
df.drop_duplicates(subset='content', inplace=True)
df.dropna(subset=['content'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Tampilkan jumlah akhir
print(f"Jumlah data setelah bersih: {len(df)}")

# Simpan ke file
df[['userName', 'score', 'content']].to_csv("shopee_reviews_clean.csv", index=False)


Mulai scraping data...


100%|██████████| 11000/11000 [01:08<00:00, 159.55it/s]

Jumlah data setelah bersih: 8320


In [ ]:
# Load data sebelumnya
df_existing = pd.read_csv("shopee_reviews_clean.csv")
existing_contents = set(df_existing['content'])

# Lanjut dari token terakhir (set awal = None)
token = None
new_reviews = []
needed = 10000 - len(df_existing)

print(f"Scraping tambahan untuk {needed} review...")

with tqdm(total=needed) as pbar:
    while len(new_reviews) < needed:
        result, token = reviews(
            'com.shopee.id',
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=200,
            continuation_token=token
        )
        for r in result:
            if r['content'] not in existing_contents:
                new_reviews.append(r)
                existing_contents.add(r['content'])
        pbar.update(len(result))
        time.sleep(1)

# Menggabungkan data
df_new = pd.DataFrame(new_reviews)
df_combined = pd.concat([df_existing, df_new], ignore_index=True)

# Simpan lagi
df_combined.to_csv("shopee_reviews_final.csv", index=False)
print(f"Jumlah data akhir: {len(df_combined)}")

Scraping tambahan untuk 1680 review...


13400it [01:21, 164.87it/s]

Jumlah data akhir: 10035
